# **검증 세트 구성**

In [1]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [2]:
from sklearn.model_selection import train_test_split

#test_size:전체 세트에서 몇퍼센트를 테스트 세트로 사용할 건지 결정
#훈련세트와 테스트세트를 구성
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2)
#훈련세트를 다시 훈련세트와 검증세트로 구성
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2)

print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [3]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9968727447678615
0.85


# **교차검증**

*훈련세트가 검증세트를 만들만큼 크지 않을 때, 검증을 하는 알고리즘이다.



In [4]:
from sklearn.model_selection import cross_validate

#교차검증은 주어진 훈련세트를 몇 개의 부분으로 나누어 그것들중 무작위로 검증세트를 선택해가며, 반복해서 학습하고 검증하여 결과를 반환한다.
#사이킥런에 기본적으로 포함된 교차 검증 알고리즘 cross_validate함수를 사용한다.
#cross_validate(모델 객체, 훈련세트 인풋, 훈련세트 타겟)
scores = cross_validate(dt, train_input, train_target)
print(scores)
#cross_validate함수는 파이썬의 기본 자료형인 딕셔너리 자료형으로 결과값을 반환한다.
#딕셔너리 자료형은 각각{'key':'value}쌍의 HashMap구조의 자료형이다.
#'fit_time': 각 시도마다 학습에 걸린 시간
#'score_time' : 각 시도마다 검증에 걸린 시간
#'test_score' : 각 시도마다의 score점수

{'fit_time': array([0.00869536, 0.00733614, 0.00723028, 0.00695395, 0.00718999]), 'score_time': array([0.00063848, 0.00053096, 0.00052166, 0.00053596, 0.00053763]), 'test_score': array([0.85      , 0.84615385, 0.83830606, 0.8719923 , 0.84696824])}


In [5]:
import numpy as np

#넘파이 배열의 mean(numpy배열)메소드는 주어진 numpy배열의 평균을 구해준다.
#딕셔너리 자료형을 다음과 같이 dict['key']로 사용할 경우 'key'와 쌍을 이루는 'value'값을 반환한다.
print(np.mean(scores['test_score']))

0.8506840897312504


**분할기를 사용한 교차검증**

In [6]:
from sklearn.model_selection import StratifiedKFold

#cv:분할기(splitter)를 지정해주는 매개변수(기본값:5)
#cv=(숫자) 로 할경우 주어진 훈련세트를 (숫자)만큼 나눠준다.
#또한 cv에 분할기(splitter)객체를 집어넣을 수 있는데
#분류 모델일 경우 다음과 같이 StratifiedKFold()를 사용
#회귀 모델일 경우 KFold()를 사용
#from sklearn.model_selection import KFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.8506840897312504


In [7]:
#n_splits:몇개의 폴드로 훈련세트를 나눌건지 설정
#shuffle:랜덤하게 섞을지 여부를 설정
#random_state:랜덤 시드 설정
splitter = StratifiedKFold(n_splits=10, shuffle=True)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8599232992441086


# **그리드 서치**

In [8]:
from sklearn.model_selection import GridSearchCV

#min_impurity_decrease:부모 자식 노드 간의 불순도 차이의 최솟값을 설정하여, 이 값만큼의 불순도 차이도 나지 않을 경우 노드를 분할하지 않게 규제한다.
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
#n_jobs:몇개의 코어를 사용하여 모델을 검증할 지 설정. n_jobs=2일 경우 동시에 2개의 모델을 학습하며, 다음과 같이 -1일경우 사용가능한 모든 코어를 사용해 모델을 학습한다.
#그리드 서치는 주어진 매개변수 값을 달리하며 교차 검증을 실시한다.
gs = GridSearchCV(DecisionTreeClassifier(), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
  

In [9]:
#GridSearchCV의 best_estimator_ 속성은 그리드 서치결과 최적으로 학습된 모델을 반환한다.
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.8891668270155859


In [10]:
#best_params_ 속성은 파라미터중 최적의 매개변수 값을 딕셔너리 자료형으로 반환한다.
print(gs.best_params_)

{'min_impurity_decrease': 0.0005}


In [14]:
#[0.0001일때,0.0002일때,0.0003일때,0.0004일때,0.0005일때]
#min_impurity_decrease=0.0005일때 스코어가 가장 큰 것을 볼 수 있다.
print(gs.cv_results_['mean_test_score'])

[0.859151   0.85799808 0.85722681 0.85838028 0.86203598]


In [18]:
print(range(5,20))

range(5, 20)


In [19]:
#min_samples_split:노드의 최소 샘플개수. 노드가 가진 샘플수가 이만큼도 되지 않는다면 노드를 분할하지 않는다.
#다음과 같이 여러개의 매개변수를 딕셔너리 자료형으로 주어 동시에 검증할 수도 있다.
#np.arange(start,stop,step):start부터 stop까지 step의 단위로 증가하는 수의 범위를 넘파이 배열로 반환한다.
#range(0,10,2):array([0,2,4,6,8])(stop은 포함하지 않는다.)
#range(start,stop,step):start부터 stop까지 step의 단위로 증가하는 수의 범위를 표현한다.
#ex)range(0,10,2):[0,2,4,6,8](stop은 포함하지 않는다.)
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'm

In [20]:
print(gs.best_params_)

{'max_depth': 8, 'min_impurity_decrease': 0.0008, 'min_samples_split': 42}


In [23]:
#np.max(넘파이배열):주어진 넘파이배열에서 가장 큰값을 반환한다.
print(np.max(gs.cv_results_['mean_test_score']))

0.8633832457244391


**랜덤서치**

In [25]:
from scipy.stats import uniform, randint
#scipy.stats안의 uniform과 randint클래스는 모두 균등분포에서 무작위 샘플링하는 클래스이다.

#randint(start,end):무작위 추출의 범위를 정함(end값 미포함)
rgen = randint(0, 10)
#rvs(n):n번만큼 추출해 샘플링
rgen.rvs(10)
#randint는 범위내에서 정수값을 샘플링한다.

array([0, 2, 0, 0, 1, 1, 1, 5, 6, 3])

In [27]:
#np.unique(넘파이배열):주어진 넘파이배열에서 중복된 값을 뺀, 요소들의 종류만 넘파이 배열로 반환
#ex)np.unique(array([1,1,2,2,3,3,4,4,5])) = array([1,2,3,4,5])
#return_counts:True로 할 경우 각 요소들이 몇 번 나왔는지 세어 넘파이 배열로 반환
np.unique(rgen.rvs(1000), return_counts=True)
#randint는 균등분포이므로 카운트가 나름 균등한 것을 볼 수 있다.

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([101,  88,  93,  96, 101, 105, 104,  89, 113, 110]))

In [28]:
ugen = uniform(0, 1)
ugen.rvs(10)
#uniform은 범위 내에서 실수값을 샘플링한다.

array([0.4012553 , 0.22486059, 0.65162606, 0.0794607 , 0.97327745,
       0.68175204, 0.23224869, 0.61471893, 0.31587724, 0.85528833])

위와 같은 식으로 정수/실수 값중 n개를 무작위로 뽑아서 매개변수에 넣어 검증하는 방식을 랜덤서치라고한다.

**랜덤 서치 적용**

In [29]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(), params, 
                        n_iter=100, n_jobs=-1)
gs.fit(train_input, train_target)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=None,
       

In [30]:
print(gs.best_params_)

{'max_depth': 44, 'min_impurity_decrease': 0.0008528626670050333, 'min_samples_leaf': 3, 'min_samples_split': 4}


In [31]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8629980750721848


In [32]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.8669230769230769
